In [16]:
import os, sys

import sgpp
import dproc
import sgutil
import pandas as pd
import polars as pl

print(sys.version)

from sklearn.pipeline import make_pipeline

3.12.9 (main, Mar 15 2025, 13:36:28) [GCC 13.3.0]


In [2]:
if not os.path.exists('data/train.csv'):
    !kaggle competitions download -c playground-series-s5e4
    !unzip playground-series-s5e4.zip -d data
    !rm playground-series-s5e4.zip
    !kaggle datasets download ysthehurricane/podcast-listening-time-prediction-dataset
    !unzip podcast-listening-time-prediction-dataset.zip -d data
    !rm podcast-listening-time-prediction-dataset.zip
!ls data

podcast_dataset.csv	  sample_submission.csv  train.csv
podcast_dataset_info.txt  test.csv


Archive:  podcast-listening-time-prediction-dataset.zip
  inflating: data/podcast_dataset.csv  
  inflating: data/podcast_dataset_info.txt  


In [8]:
!ls data/

sample_submission.csv  test.csv  train.csv


In [17]:
sgpp.PandasCoverter?

Init signature: sgpp.PandasCoverter(index_col=None)
Docstring:     
Mixin class for all transformers in scikit-learn.

This mixin defines the following functionality:

- a `fit_transform` method that delegates to `fit` and `transform`;
- a `set_output` method to output `X` as a specific container type.

If :term:`get_feature_names_out` is defined, then :class:`BaseEstimator` will
automatically wrap `transform` and `fit_transform` to follow the `set_output`
API. See the :ref:`developer_api_set_output` for details.

:class:`OneToOneFeatureMixin` and
:class:`ClassNamePrefixFeaturesOutMixin` are helpful mixins for
defining :term:`get_feature_names_out`.

Examples
--------
>>> import numpy as np
>>> from sklearn.base import BaseEstimator, TransformerMixin
>>> class MyTransformer(TransformerMixin, BaseEstimator):
...     def __init__(self, *, param=1):
...         self.param = param
...     def fit(self, X, y=None):
...         return self
...     def transform(self, X):
...         return n

In [3]:
p = make_pipeline(
    sgpp.PolarsProcessor(),
    sgpp.PandasConverter(index_col = 'id')
)
p.fit(['data/train.csv'])

Pipeline(steps=[('polarsprocessor',
                 <sgpp.PolarsProcessor object at 0x7f736c7a6420>),
                ('pandasconverter',
                 <sgpp.PandasConverter object at 0x7f73c9c46150>)])

In [7]:
df_train = p.transform(['data/train.csv'])
df_test = p.transform(['data/test.csv'])
df_org = p.transform(['data/podcast_dataset.csv'])

In [20]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 750000 entries, 0 to 749999
Data columns (total 11 columns):
 #   Column                       Non-Null Count   Dtype   
---  ------                       --------------   -----   
 0   Podcast_Name                 750000 non-null  category
 1   Episode_Title                750000 non-null  category
 2   Episode_Length_minutes       662907 non-null  float32 
 3   Genre                        750000 non-null  category
 4   Host_Popularity_percentage   750000 non-null  float32 
 5   Publication_Day              750000 non-null  category
 6   Publication_Time             750000 non-null  category
 7   Guest_Popularity_percentage  603970 non-null  float32 
 8   Number_of_Ads                749999 non-null  float32 
 9   Episode_Sentiment            750000 non-null  category
 10  Listening_Time_minutes       750000 non-null  float32 
dtypes: category(6), float32(5)
memory usage: 21.5 MB


In [25]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 250000 entries, 750000 to 999999
Data columns (total 10 columns):
 #   Column                       Non-Null Count   Dtype   
---  ------                       --------------   -----   
 0   Podcast_Name                 250000 non-null  category
 1   Episode_Title                250000 non-null  category
 2   Episode_Length_minutes       221264 non-null  float32 
 3   Genre                        250000 non-null  category
 4   Host_Popularity_percentage   250000 non-null  float32 
 5   Publication_Day              250000 non-null  category
 6   Publication_Time             250000 non-null  category
 7   Guest_Popularity_percentage  201168 non-null  float32 
 8   Number_of_Ads                250000 non-null  float32 
 9   Episode_Sentiment            250000 non-null  category
dtypes: category(6), float32(4)
memory usage: 6.2 MB


In [26]:
df_org.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52500 entries, 0 to 52499
Data columns (total 11 columns):
 #   Column                       Non-Null Count  Dtype   
---  ------                       --------------  -----   
 0   Podcast_Name                 52500 non-null  category
 1   Episode_Title                52500 non-null  category
 2   Episode_Length_minutes       47254 non-null  float32 
 3   Genre                        52500 non-null  category
 4   Host_Popularity_percentage   52500 non-null  float32 
 5   Publication_Day              52500 non-null  category
 6   Publication_Time             52500 non-null  category
 7   Guest_Popularity_percentage  47242 non-null  float32 
 8   Number_of_Ads                52500 non-null  float32 
 9   Episode_Sentiment            52500 non-null  category
 10  Listening_Time_minutes       47105 non-null  float32 
dtypes: category(6), float32(5)
memory usage: 1.3 MB


In [21]:
X_num = ['Episode_Length_minutes', 'Host_Popularity_percentage', 'Guest_Popularity_percentage', 'Number_of_Ads']
X_cat = ['Episode_Sentiment', 'Episode_Title', 'Genre', 'Podcast_Name', 'Publication_Day', 'Publication_Time']
target = 'Listening_Time_minutes'

In [10]:
df_train[X_cat].apply(lambda x: x.nunique())

Episode_Sentiment      3
Episode_Title        100
Genre                 10
Podcast_Name          48
Publication_Day        7
Publication_Time       4
dtype: int64

In [12]:
df_test[X_cat].apply(lambda x: x.nunique())

Episode_Sentiment      3
Episode_Title        100
Genre                 10
Podcast_Name          48
Publication_Day        7
Publication_Time       4
dtype: int64

In [19]:
df_org[X_cat].apply(lambda x: x.nunique())

Episode_Sentiment      3
Episode_Title        100
Genre                 10
Podcast_Name          48
Publication_Day        7
Publication_Time       4
dtype: int64

In [17]:
# Train에 등장하지 않는 케이스가 Test에 등장하는가?
pd.concat([
    df_train[X_cat].apply(set).rename('train'),
    df_test[X_cat].apply(set).rename('test')
], axis=1).apply(
    lambda x: x['test'] - x['train'], axis=1
)

Episode_Sentiment    {}
Episode_Title        {}
Genre                {}
Podcast_Name         {}
Publication_Day      {}
Publication_Time     {}
dtype: object

In [ ]:
# Train에 등장하지 않는 케이스가 Test에 등장하는가?
pd.concat([
    df_train[X_num].apply(set).rename('train'),
    df_test[X_cat].apply(set).rename('test')
], axis=1).apply(
    lambda x: x['test'] - x['train'], axis=1
)

In [22]:
df_train[X_num].apply(lambda x: x.nunique())

Episode_Length_minutes         12203
Host_Popularity_percentage      8038
Guest_Popularity_percentage    10019
Number_of_Ads                     12
dtype: int64

In [23]:
df_test[X_num].apply(lambda x: x.nunique())

Episode_Length_minutes         11620
Host_Popularity_percentage      8010
Guest_Popularity_percentage     9961
Number_of_Ads                      6
dtype: int64

In [24]:
df_org[X_num].apply(lambda x: x.nunique())

Episode_Length_minutes         11297
Host_Popularity_percentage      7976
Guest_Popularity_percentage     9899
Number_of_Ads                      4
dtype: int64